In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
input_path = '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/inter_dataset'
total_count = 0

datasets = os.listdir(input_path)
counts = []
df_list = []

for input_file in os.listdir(input_path):
    df = pd.read_parquet(os.path.join(input_path, input_file))
    df_list.append(df)
    counts.append(len(df))

count_df = pd.DataFrame({'dataset': datasets, 'count': counts}
                        ).sort_values('count', ascending=False)

all_edge_df = pd.concat(df_list)
print(count_df['count'].sum())

In [ ]:
all_edge_df

In [ ]:
all_edge_df[all_edge_df['cell_type_ref'] == 'Border-associated macrophages']['cell_type_query'].value_counts()

In [ ]:
all_edge_df[(all_edge_df['cell_type_ref'] == 'Monocytes')
        & (all_edge_df['cell_type_query'] == 'monocyte')]

In [ ]:
for i, f in enumerate(sorted(os.listdir(input_path))):
    df = pd.read_parquet(os.path.join(input_path, f))
    print(f'{i}: {f} : {len(df)}')
    

In [ ]:
count_df[count_df['dataset'].str.contains('Suo_ImmuneDev')]

In [ ]:
def sigmoid(x, center, scale):
    return 1 / (1 + np.exp(-scale * (x - center)))

sigmoid(x=1.2, center=1, scale=-10)

In [ ]:
input_file = os.path.join(
    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/inter_dataset',
    'Qiu_Organogenesis_MM_2022_all_AND_Qiu_whole_embryo_dev_MM_2024_all.parquet')
#input_file = os.path.join(
#    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_dataset',
#    'Park_Thymus_HS_2020_all.parquet')

mkn_df = pd.read_parquet(input_file)

print(mkn_df.shape[0])

cross_df = pd.crosstab(
    mkn_df['cell_type_ref'].values,
    mkn_df['cell_type_query'].values,
)

for ct2 in sorted(cross_df.columns):

    print(f'\n{ct2}:')
    for ct1 in cross_df[ct2].sort_values(ascending=False).index[:5]:
        if cross_df.loc[ct1, ct2] > 0:
            local_df = mkn_df[(mkn_df['cell_type_ref'] == ct1) & (mkn_df['cell_type_query'] == ct2)]

            print(f'{cross_df.loc[ct1, ct2]}, \t{ct1}')
    
    #if ct in cross_df_filtered.columns:
    #    display(cross_df_filtered[ct].sort_values(ascending=False).head(5))

In [ ]:
mkn_df['dist'].hist(bins=100)

In [ ]:
cross_df['Epiblast'].sort_values(ascending=False)[:20]

In [ ]:
import json
import numpy as np

with open('integration_specs.json', 'r') as f:
    i_spec_dict = json.load(f)

dataset_ids = []
for k in i_spec_dict:
    dataset_ids.append(k)
    for v in i_spec_dict[k]:
        if k == v:
            print(f'Error: {k} cannot integrate with itself')
        dataset_ids.append(v)

dataset_ids = list(np.unique(dataset_ids))

# Sort the dataset IDs by the number of integration events
dataset_counts = np.zeros(len(dataset_ids), dtype=int)
for k in i_spec_dict:
    for v in i_spec_dict[k]:
        dataset_counts[dataset_ids.index(k)] += 1
        dataset_counts[dataset_ids.index(v)] += 1

for i in dataset_counts.argsort()[::-1]:
    print(f'{dataset_ids[i]}: {dataset_counts[i]}')

print(dataset_counts.sum() / 2)